In [1]:
import os
from azureml.core import  Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.environment import Environment 
from azureml.core.webservice import AciWebservice,Webservice
from azureml.core.model import Model,InferenceConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, DEFAULT_GPU_IMAGE

interactive_auth = InteractiveLoginAuthentication(tenant_id="9ce70869-60db-44fd-abe8-d2767077fc8f")
ws = Workspace.from_config()

In [2]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: cdh-azml-dev-mlw
Azure region: eastus
Subscription id: 320d8d57-c87c-4434-827f-59ee7d86687a
Resource group: csels-cdh-dev


In [20]:
model = Model(workspace=ws,name='dan_d1_misa_pt',version=2)
print(os.getenv("AZUREML_MODEL_DIR"))
#model_path = Model.get_model_path(os.getenv("AZUREML_MODEL_DIR").split('/')[-2])

None


In [21]:
import os
os.makedirs('./aml_outputs/',exist_ok=True)
model.download(target_dir='./aml_outputs/',exist_ok=True)

'aml_outputs/dan_d1_misa_pt'

In [12]:
from tensorflow import keras
import tensorflow as tf
loaded_model = keras.models.load_model('aml_outputs/dan_d1_misa_pt.h5/')

In [22]:
import joblib
from tensorflow import keras
import tensorflow as tf

# tensorflow version must match training version
loaded_model = keras.models.load_model('aml_outputs/dan_d1_misa_pt/',custom_objects={'tf':tf},compile=True)

In [6]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [31]:
from azureml.core import Workspace, Dataset
from azureml.core import Run
data_store = ws.get_default_datastore()

##########Loading the data from datastore
print("Creating dataset from Datastore")
inputs = Dataset.File.from_files(path=data_store.path('pkl/trimmed_seqs.pkl'))
vocab = Dataset.File.from_files(path=data_store.path('pkl/all_ftrs_dict.pkl'))
demog_dict = Dataset.File.from_files(path=data_store.path('pkl/demog_dict.pkl'))

Creating dataset from Datastore


In [32]:
inputs.download(target_path='./',overwrite=True,ignore_not_found=True)
vocab.download(target_path='./',overwrite=True,ignore_not_found=True)
demog_dict.download(target_path='./',overwrite=True,ignore_not_found=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/wsn8-su2/code/Users/WSN8-SU/premier_analysis/azure_ml/demog_dict.pkl']

In [33]:
import pickle as pkl
with open("trimmed_seqs.pkl", "rb") as f:
    inputs = pkl.load(f)

with open(os.path.join("all_ftrs_dict.pkl"), "rb") as f:
        vocab = pkl.load(f)

with open(os.path.join("demog_dict.pkl"), "rb") as f:
        demog_lookup = pkl.load(f)

In [35]:
import numpy as np

In [36]:
features = [l[0][-1] for l in inputs]
N_VOCAB = len(vocab) + 1
N_DEMOG = len(demog_lookup) + 1

In [37]:
N_VOCAB,N_DEMOG

(44727, 18)

In [38]:
new_demog = [[i + N_VOCAB - 1 for i in l[1]] for l in inputs]
features = [
                features[i] + new_demog[i] for i in range(len(features))
            ]
demog_vocab = {k: v + N_VOCAB - 1 for k, v in demog_lookup.items()}
vocab.update(demog_vocab)
N_VOCAB = np.max([np.max(l) for l in features]) + 1

In [39]:
import tensorflow.keras as keras
X = keras.preprocessing.sequence.pad_sequences(features,padding='post')
X.shape

(67562, 1895)

In [49]:
test = X[0]
test = np.expand_dims(test,axis=0)
test.shape

(1, 1895)

In [50]:
preds = loaded_model.predict(test)
preds.shape

1/1 [==============================] - 0s 20ms/step


(1, 1)

## Deploying Model

In [51]:
preds[0]

array([0.06474929], dtype=float32)

In [2]:
%%writefile conda_dependencies_score.yml
channels:
- anaconda
- default
dependencies:
- python=3.8
- pip:
  - azureml-defaults
  - matplotlib
  - pandas
  - argparse
  - joblib
  - inference-schema[numpy-support]
  - scikit-learn
  - azureml-sdk
  - openpyxl
  - tensorflow

Overwriting conda_dependencies_score.yml


In [ ]:
premier_train_score_env = Environment.from_conda_specification(name='premier_train_model_env', file_path='conda_dependencies_score.yml')
# Specify a CPU base image
# premier_train_model_env.docker.enabled = True
premier_train_score_env.docker.base_image = DEFAULT_CPU_IMAGE
premier_train_score_env.register(workspace=ws)


#### Create Inference Configuration

* file_path: input parameter to Environment constructor. Manages conda and python package dependencies.
* env.docker.base_dockerfile: any extra steps you want to inject into docker file
* source_directory: holds source path as string, this entire folder gets added in image so its really easy to access any files within this folder or subfolder
* entry_script: contains logic specific to initializing your model and running predictions

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

MOD_NAME = 'dan'
OUTCOME = 'misa_pt'
service_name = f"premier-model-{MOD_NAME}-{OUTCOME}"

inference_config = InferenceConfig(entry_script='./training/score.py', environment=premier_train_score_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=1 )


service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)